<a href="https://colab.research.google.com/github/WittyTheMighty/ML-kaggle/blob/main/final_5_decembre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Final Kaggle competition ML

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install wandb --upgrade

This is how we can identify the model.

In [ ]:
#Exemple d'utilisation 

# import wandb
# # At the top of your training script, start a new run
# wandb.init(project="test-project", entity="xabjuwplb")
# # Capture a dictionary of hyperparameters with config
# wandb.config = {
#   "learning_rate": 0.001,
#   "epochs": 100,
#   "batch_size": 128
# }

# # Log metrics inside your training loop to visualize model performance
# wandb.log({"loss": loss})

# # Optional
# wandb.watch(model)


###Import: 

In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import torch as torch
import torchvision as  tv
import torch.nn as nn
import torchvision.transforms as transforms
import os
import pandas as pd
import pickle as pkl
from matplotlib import cm
from PIL import Image
import torch.optim as optim
import torch.nn as nn 
import torch.nn.functional as F 
import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import csv
import scipy as sp
import copy

import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.utils as vutils
device = torch.device('cuda')

from sklearn.utils import shuffle
import os
import random

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import wandb
from sklearn.metrics import f1_score
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration


######Weigth and biais imports

To track our model we use weights and biaises a plateform that allow us to track the data in the pipeline.

In [ ]:
#PATH = './drive/MyDrive/ML-Kaggle/'
PATH = "/content/drive/MyDrive/ML-Kaggle/"
# PATH = '../data/'

#Load
x_data = np.array(pkl.load(open(PATH+'x_train.pkl', 'rb')))
y_data = np.array(pkl.load(open(PATH+'y_train.pkl', 'rb')))

x_test_submission =  np.array(pkl.load(open(PATH+'x_test.pkl', 'rb')))



In [ ]:
x_train = x_data[:10000]
y_train = y_data[:10000]

#Before submit we test the model on this subsection of the dataset without augmented data
x_valid = x_data[10001:]
y_valid = y_data[10001:]

assert len(x_train) == len(y_train)
assert len(x_valid) == len(y_valid)

In [ ]:
labels = np.unique(y_train)
labels = dict(zip(labels, range(len(labels))))

In [ ]:
labels = np.unique(y_valid)
labels=dict(zip(labels, range(len(labels))))

---

## Preprocessing

####Data augmentation from strach

In [ ]:
#First preprocessing step 

#Utilitary function for geometric preprocessing
# def flip_verticaly(X):
#     flipped_image = copy.deepcopy(rotate90(X))
#     for idx,channel in enumerate(flipped_image):
#         flipped_image[idx]= np.fliplr(channel)

#     return rotate(flipped_image, 270)


def flip_horizontaly(X):
    flipped_image = copy.deepcopy(X)
    flipped_image= np.fliplr(flipped_image)

    return flipped_image

def rotate90(X):
    rot90_img = copy.deepcopy(X)
    rot90_img= np.rot90(rot90_img)
    return rot90_img

def rotate(X,degree):
    rot_img = copy.deepcopy(X)
    rot_img= sp.ndimage.rotate(rot_img, degree, reshape=False)
    return rot_img

#An adaptation from:
# https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions
def clipped_zoom(img, zoom_factor):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = sp.ndimage.zoom(img, zoom_tuple)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = sp.ndimage.zoom(img[top:top+zh, left:left+zw], zoom_tuple)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

def blur(X,blur_factor):
    blur_img = copy.deepcopy(X)
    blur_img= sp.ndimage.gaussian_filter(blur_img, sigma=blur_factor)
    return blur_img


In [ ]:
zoom_factors = [1.1, 1.3, 1.5]
x_train_zoomed_vertical = np.array([clipped_zoom(x,np.random.choice(zoom_factors)) for x in x_train])
x_train_zoomed_horizontal = np.array([rotate(clipped_zoom(rotate90(x),np.random.choice(zoom_factors)),270) for x in x_train])

In [ ]:
x_train_rotated = np.array([rotate(x,round(np.random.random()*360)) for x in x_train])

In [ ]:
x_train_flipped_horizontally = np.array([flip_horizontaly(x) for x in x_train])

In [ ]:
augmented_x_train = np.concatenate([x_train, x_train_zoomed_horizontal, x_train_zoomed_vertical, x_train_rotated, x_train_flipped_horizontally], axis=0)

In [ ]:
augmented_y_train = np.concatenate([y_train]*5)
augmented_y_train.shape

X = np.array([[1., 0.], [2., 1.], [0., 0.]])
y = np.array([0, 1, 2])

X_train, y_train= shuffle(augmented_x_train, augmented_y_train, random_state=0)

In [ ]:
# SAVE AUGMENTED TRAINING SET
pkl.dump(augmented_x_train,open(PATH+'x_train_aug.pkl', 'wb'))
pkl.dump(augmented_y_train,open(PATH+'y_train_aug.pkl', 'wb'))

In [ ]:
# LOAD AUGMENTED TRAINING SET
x_train = pkl.load(open(PATH+'x_train_aug.pkl', 'rb'))
y_train = pkl.load(open(PATH+'y_train_aug.pkl', 'rb'))

###Normalisation

From strach not used yet. We use pytorch normalize function.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

def standardize(data):
    standardized_data = []
    scaler = StandardScaler()
    for img in data:
        scaler = scaler.fit(img)
        standadized_data.append(scaler.transform(img))
    return np.array(standardized_data)

def normalize(data):
    normalized_data = []
    normalizer = Normalizer()
    for img in data:
        normalizer = normalizer.fit(img)
        normalized_data.append(normalizer.transform(img))
    return np.array(normalized_data)




In [ ]:

# x_train_normalize = normalize(x_train)

# #Todo : Normaliser les données de tests

# x_test_set = x_train_normalize[:3000]
# y_test_set = y_train[:3000]

# x_train = x_train_normalize[3000:]
# y_train = y_train[3000:]

# assert y_train.shape[0] == x_train.shape[0]
# assert y_test_set.shape[0] == x_test_set.shape[0]

---
## Models

This is all the model we used in our expirements.

##### Inspired AlexNet model

In [ ]:

class AlexNet(nn.Module):
    def __init__(self): 
        super().__init__()
        print("AlexNet")
        self.conv1 = nn.Conv2d(1,25,5)
        self.batchNorm2d1 = nn.BatchNorm2d(25,momentum=0.95)
        # we use the maxpool multiple times, but define it once
        self.pool = nn.MaxPool2d(2, 2)
        # in_channels = 6 because self.conv1 output 6 channel
        self.conv2 = nn.Conv2d(25, 50,5)
        self.batchNorm2d2 = nn.BatchNorm2d(50,momentum=0.95)
        self.conv3 = nn.Conv2d(50 ,100,3)
        self.batchNorm2d3 = nn.BatchNorm2d(100,momentum=0.95)
        # 5*5 comes from the dimension of the last convnet layer
        self.fc1 = nn.Linear(100*9*9, 480) 
        self.fc2 = nn.Linear(480, 240)
        self.fc3 = nn.Linear(240, 120)
        self.fc4 = nn.Linear(120, 11)
        
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.batchNorm2d1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.batchNorm2d2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.batchNorm2d3(x)
        x = x.view(-1, 100*9*9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)  # no activation on final layer 
        return x


class AlexNetv2(nn.Module): 
    def __init__(self): 
        super().__init__()
        print("AlexNetv2")
        self.convolution = nn.Sequential(
            nn.Conv2d(1, 48,7), #in, out, kernel_size, stride,padding
            nn.MaxPool2d(2,stride=2), #kernel_size
            nn.ReLU(),
            nn.BatchNorm2d(48),
            #Layer 2
            nn.Conv2d(48, 128,5,stride=2), #(45-5)/2 +1 = 21
            nn.MaxPool2d(3,2),
            nn.ReLU(),
            nn.BatchNorm2d(128,momemtum=0.),
            nn.Conv2d(128, 192, 5,padding=2), 
            nn.ReLU(),
            nn.Conv2d(192, 192, 3,padding=1), #(39-3+1)/1 =37
            nn.ReLU(),
            nn.Conv2d(192, 128, 3, 1), # 35
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1), 
            nn.MaxPool2d(2,stride=2),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            #3200 = 5 * 5 *128
            nn.Linear(128*3*3, 4096),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(4096, 11),
        )
        
    def forward(self, x): 
        x = self.convolution(x)
        x = x.view(-1,128*3*3)
        x = self.classifier(x)
        return x
    


    



#### Inspired ResNet

In [ ]:
class block(nn.Module):
        def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
            super(block, self).__init__()
            self.expansion = 4
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
            self.bn1 = nn.BatchNorm2d(out_channels)
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
            self.bn2 = nn.Batch+ample = identity_downsample
        
        def forward(self, x):
            identity = x
            
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.conv2(x)
            x = self.bn2(x)
            x = self.relu(x)
            x = self.conv3(x)
            x = self.bn3(x)
            
            if self.identity_downsample is not None:
                 identity = self.identity_downsample(identity)
                    
            x += identity
            x = self.relu(x)
            return x

class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet layers
        self.layer1 = self._make_layer_(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer_(block, layers[1], out_channels=128, stride = 2)
        self.layer3 = self._make_layer_(block, layers[2], out_channels=256, stride = 2)
        self.layer4 = self._make_layer_(block, layers[3], out_channels=512, stride = 2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*4, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x) 
        x = x.reshape(x.shape[0], -1)
        
        x = self.fc(x)
        
        return x
        
    def _make_layer_(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1, stride=stride), nn.BatchNorm2d(out_channels*4))
            
        layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels*4
        
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)
    
def ResNet50(img_channels=1, num_classes=11):
    return ResNet(block, [3,4,6,3], img_channels, num_classes)

    
def ResNet152(img_channels=1, num_classes=11):
    return ResNet(block, [3,8,36,3], img_channels, num_classes)

def test():
        net = ResNet50()
        x = torch.randn(2, 1, 96, 96)
        y = net(x).to("cuda")
        print(y.shape)
        
test()

#### Inspired VGG

In [ ]:
# https://debuggercafe.com/implementing-vgg11-from-scratch-using-pytorch/
# the VGG11 architecture
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        # convolutional layers 
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # fully connected linear layers
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=4608, out_features=4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(in_features=4096, out_features=11)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        # flatten to prepare for the fully connected layers
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

net = VGG()
x = torch.randn(2, 1, 96, 96)
y = net(x).to("cuda")

##Pipeline

For our pipeline we used the pipeline framework weigths and biaises for easy and fast data parameter tracking. All the result of our experiments will be store on the cloud. 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cuda')
#device = torch.device("cpu")

from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.utils as vutils

class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms."""
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]
        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)
    


#####Custom TensorDataset

In [ ]:
#Creating dataloader object
trans = transforms.Compose(
    [transforms.Normalize((0.5), (0.5))]
)


#Training set
x_train_tensor = torch.tensor(x_train,  dtype=torch.float32)[:, None, :, :]
y_train_tensor = torch.tensor(np.array([labels[y] for y in y_train]),  dtype=torch.long)

train_dataset = CustomTensorDataset(tensors=(x_train_tensor, y_train_tensor), transform=trans)





#Validation set
x_valid_tensor = torch.tensor(x_valid,  dtype=torch.float32)[:, None, :, :]
y_valid_tensor = torch.tensor(np.array([labels[y] for y in y_valid]),  dtype=torch.long)

valid_dataset = CustomTensorDataset(tensors=(x_valid_tensor, y_valid_tensor), transform=trans)


In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,  pin_memory = True, batch_size=64, shuffle=True)

####Hyperparameter list
 
If new variable added but them in the config file

Supported models: AlexNet

```

hyperparam = {
    "training_data": train_data,
    "valid_data": valid_data,

    "model":"AlexNet",
    "learning_rate":1e-2,
    "batch_size":64,
    "epochs": 5,

    "k_folds" :5,
    "description":"Setting up momemtum to 0.95"
    }
```


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#This code is an adaptation from Weigth&BiaisesFramework we established a pipeline to track our data : https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Simple_PyTorch_Integration.ipynb#scrollTo=bZiTlrNkRKzm&uniqifier=1
def model_pipeline(hyperparameters,training_data,valid_data,PATH):
    """"
    "Hyperparameters need to be a dictionary with all the necessary information" Wandb librairie will load the data"
    "into their server."
    """
    # tell wandb to get started
    with wandb.init(entity="xabjuwplb",project="CNN", config=hyperparameters):

        config = wandb.config
        train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=config.batch_size, shuffle=True)
        valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, shuffle=True)

        # make the model, data, and optimization problem
        model, criterion, optimizer = make(config)
        model.to(device)
        criterion.to(device)
        # and use them to train the model
        train(model, train_dataloader,valid_dataloader, criterion, optimizer, config)

        # and test its final performance
        test(model, valid_dataloader,train_dataloader)
        torch.save(model.state_dict(), PATH)

    return model

def make(config):

    # Make the model
    model = initialize_model(config)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(
        model.parameters(), lr=config.learning_rate)    
    return model, criterion, optimizer

def make_loaders(config):

    return train_dataloader, valid_dataloader


def initialize_model(config):
    #When implmenting add it to the map
    if config.model == "AlexNet":
        return AlexNet()
    elif config.model == "AlexNetv2":
        return AlexNetv2()

def train(model, train_loader,valid_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)
    # Run training and track with wandb
    total_batches = len(train_loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    loss = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (image,label) in enumerate(train_loader):
            loss = train_batch(image, label, model, optimizer, criterion)
            valid_loss = valid_batch(valid_loader,model,criterion)
            example_ct +=  len(image)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss,valid_loss, example_ct, epoch)
        test(model,valid_loader,train_loader)
    

#
def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass ➡

    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()
    # Step with optimizer
    optimizer.step()
    return loss
def valid_batch(valid_data_load,model,criterion):

    valid_loss = 0
    valid_image,valid_label = next(iter(valid_data_load))

    valid_images, valid_labels = valid_image.to(device), valid_label.to(device)
    # Forward pass ➡
    outputs = model(valid_images)

    valid_loss += criterion(outputs, valid_labels)

    return valid_loss



def test(model, test_loader,train_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        true_y = np.array([])
        predicted_y = np.array([])
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # predicted_y = np.append(predicted_y,predicted.cpu().detach().tolist())
            # true_y = np.append(true_y,labels.cpu().detach().tolist())

        # f1_score_micro = f1_score(true_y,predicted_y,average="micro")
        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")

        
        wandb.log({"test_accuracy": correct / total})
    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        true_y =np.array([])
        predicted_y = np.array([])
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # true_y= np.append(true_y,labels.cpu().detach().tolist())
            # predicted_y=np.append(predicted_y,predicted.cpu().detach().tolist())    

        print(f"Accuracy of the model on the {total} " +
              f"training images: {100 * correct / total}%")
        wandb.log({"training_accuracy": correct / total})
        # f1_score_micro = f1_score(true_y,predicted_y,average="micro")
        # print(f"F1 micro score {total} " +
        #       f"training images: {f1_score_micro}")        
        # wandb.log({"training_accuracy": correct / total})


In [ ]:
def train_log(loss,valid_loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "training_loss": loss,"valid_loss":valid_loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")
    print(f"Validation loss after " + str(example_ct).zfill(5) + f" examples: {valid_loss:.3f}")
    

In [ ]:
def save_model(epoch,model,optimizer,loss,path):

    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "loss": loss
    }, path)
###
#
def load_model(checkpoints_path):
    checkpoints = torch.load(PATH)
    with wandb.init(entity="xabjuwplb",project="CNN", config=checkpoints):
        config = wandb.config 
        #Initialize
        model = initialize_model(config.model)
        optimizer = nn.CrossEntropyLoss()


        optimizer = optimizer.load_state_dict(config['optimizer_state_dict'])
        checkpoint = torch.load(PATH)
        model.load_state_dict(config['model_state_dict'])
        optimizer.load_state_dict(config['optimizer_state_dict'])
        epoch = config['epoch']
        loss = config['loss']


In [ ]:
config = {

    "model":"AlexNetv2",
    "learning_rate":1e-2,
    
    "batch_size":32,
    "epochs": 12,
    "description": ""
    }


### Running the models

In [ ]:
# Build, train and analyze the model with the pipeline
model = model_pipeline(config,train_dataset,valid_dataset,"model.pth")

AlexNetv2


  0%|          | 0/12 [00:00<?, ?it/s]

Loss after 00768 examples: 2.394
Validation loss after 00768 examples: 2.398
Loss after 01568 examples: 2.378
Validation loss after 01568 examples: 2.365
Loss after 02368 examples: 2.369
Validation loss after 02368 examples: 2.383
Loss after 03168 examples: 2.390
Validation loss after 03168 examples: 2.373
Loss after 03968 examples: 2.368
Validation loss after 03968 examples: 2.325
Loss after 04768 examples: 2.340
Validation loss after 04768 examples: 2.359
Loss after 05568 examples: 2.378
Validation loss after 05568 examples: 2.321
Loss after 06368 examples: 2.355
Validation loss after 06368 examples: 2.313
Loss after 07168 examples: 2.352
Validation loss after 07168 examples: 2.331
Loss after 07968 examples: 2.323
Validation loss after 07968 examples: 2.392
Loss after 08768 examples: 2.302
Validation loss after 08768 examples: 2.352
Loss after 09568 examples: 2.264
Validation loss after 09568 examples: 2.343
Loss after 10368 examples: 2.335
Validation loss after 10368 examples: 2.322

#### Soumission

In [ ]:
def make_submission(x_test, network):
    header = ["Id", "class"]
    id_value = 0

    classifications =[]
    for image in x_test:
        image = torch.reshape(image[0], (1,1,96, 96))
        classifications.append([id_value, network(image.float()).argmax(dim=1)[0].item()])
        id_value += 1

    y = PATH+"./submission_3dec.csv"
    with open(y, 'w', newline='') as work:
        z = csv.writer(work)
        z.writerow(header)
        z.writerows(classifications)

In [ ]:
#loading data
x_test_scalar =  np.array(pkl.load(open(PATH+'x_test.pkl', 'rb')))
x_test = x_test_scalar

x_test = torch.tensor(x_test, dtype=float)[:, None, :, :].to(device)
y_test = torch.tensor(np.array([1 for y in x_test]), dtype=float)

trans = transforms.Compose(
    [transforms.Normalize((0.5), (0.5))]
)
#Transformation
test_dataset = CustomTensorDataset(tensors=(x_test, y_test), transform=trans)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                          shuffle=False)
#Insert model
network = AlexNetv2().to(device)
network.load_state_dict(torch.load('./model.pth'))
network.eval()

make_submission(test_dataloader, network)

AlexNetv2
